<a href="https://colab.research.google.com/github/yukinaga/hopfield_boltzmann/blob/main/section_3/01_boltzmann_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ボルツマンマシンをPythonのクラスで実装する
まずは、「ボルツマンマシン」のPythonのクラスによる実装を解説します。

## BoltzmannMachineクラス
ボルツマンマシンをクラスとして実装します。

In [ ]:
import numpy as np

class BoltzmannMachine:
    def __init__(self, num_neurons, temperature=1.0):
        """ボルツマンマシンの初期化"""
        self.num_neurons = num_neurons
        self.weights = np.random.randn(num_neurons, num_neurons) * 0.1
        np.fill_diagonal(self.weights, 0)  # 自己結合はゼロ
        self.biases = np.zeros(num_neurons)
        self.temperature = temperature  # 温度パラメータ

    def sigmoid(self, x):
        """シグモイド関数（温度パラメータを考慮）"""
        return 1 / (1 + np.exp(-x / self.temperature))

    def energy(self, state):
        """エネルギー関数"""
        return -0.5 * np.dot(state, np.dot(self.weights, state)) - np.dot(self.biases, state)

    def sample_neuron(self, state, idx):
        """単一のニューロンの状態を確率的に更新"""
        activation = np.dot(self.weights[idx], state) + self.biases[idx]
        prob = self.sigmoid(activation)
        return 1 if np.random.rand() < prob else 0

    def sample_state(self, state, steps=1):
        """全ニューロンの状態をランダムに更新"""
        for _ in range(steps):
            for i in range(self.num_neurons):
                state[i] = self.sample_neuron(state, i)
        return state

    def train(self, data, epochs=1000, learning_rate=0.01):
        """ボルツマンマシンの学習"""
        num_samples = data.shape[0]
        for epoch in range(epochs):
            for sample in data:
                positive_state = sample.copy()
                negative_state = sample.copy()

                # 負方向のフェーズ（ギブスサンプリング）
                for _ in range(10):
                    negative_state = self.sample_state(negative_state)

                # 重みとバイアスの更新
                for i in range(self.num_neurons):
                    for j in range(self.num_neurons):
                        self.weights[i, j] += learning_rate * (
                            positive_state[i] * positive_state[j] - negative_state[i] * negative_state[j]
                        )
                self.biases += learning_rate * (positive_state - negative_state)

            # 温度を徐々に下げる（アニーリング）
            self.temperature *= 0.99

            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}/{epochs} completed, Temperature: {self.temperature:.4f}")

    def run(self, state, steps=10):
        """学習済みボルツマンマシンでデータを生成"""
        return self.sample_state(state, steps)


### **クラスの概要**
このクラス `BoltzmannMachine` は、ボルツマンマシン (Boltzmann Machine) を表現しています。  
ボルツマンマシンは、ニューラルネットワークの一種で、特にエネルギーベースモデル (Energy-based Model) として知られています。  
これは、ノード（ニューロン）同士が相互に接続され、確率的にデータのパターンを学習し再生成することを目的としています。

---

### **クラスの初期化 (`__init__` メソッド)**
```python
def __init__(self, num_neurons, temperature=1.0):
    """ボルツマンマシンの初期化"""
    self.num_neurons = num_neurons
    self.weights = np.random.randn(num_neurons, num_neurons) * 0.1
    np.fill_diagonal(self.weights, 0)  # 自己結合はゼロ
    self.biases = np.zeros(num_neurons)
    self.temperature = temperature  # 温度パラメータ
```
- **`num_neurons`**: ニューロンの数（ネットワーク内のノードの数）です。ここでは64個（8×8の画像パターン）が使用されます。
- **`temperature`**: 温度パラメータ。高い温度では探索的になり、低い温度では確定的な動作になります。
- **`weights`**: 各ニューロン間の重み行列。初期化時にはランダムな値（標準正規分布に0.1を掛けたもの）を設定します。
- **`biases`**: 各ニューロンのバイアス値。初期化時はゼロです。
- **`np.fill_diagonal(self.weights, 0)`**: 自己結合（同じニューロン同士の重み）をゼロに設定します。

---

### **シグモイド関数 (`sigmoid` メソッド)**
```python
def sigmoid(self, x):
    """シグモイド関数（温度パラメータを考慮）"""
    return 1 / (1 + np.exp(-x / self.temperature))
```
- シグモイド関数は、入力値を0から1の範囲の確率に変換します。
- 温度パラメータ `self.temperature` が確率の決定に影響します。

---

### **エネルギー関数 (`energy` メソッド)**
```python
def energy(self, state):
    """エネルギー関数"""
    return -0.5 * np.dot(state, np.dot(self.weights, state)) - np.dot(self.biases, state)
```
- **エネルギー関数**は、ネットワークの状態の「安定度」を測る指標です。エネルギーが低いほど安定した状態とみなされます。

---

### **単一のニューロンをサンプリング (`sample_neuron` メソッド)**
```python
def sample_neuron(self, state, idx):
    """単一のニューロンの状態を確率的に更新"""
    activation = np.dot(self.weights[idx], state) + self.biases[idx]
    prob = self.sigmoid(activation)
    return 1 if np.random.rand() < prob else 0
```
- 特定のニューロン `idx` の状態を更新します。
- 重みとバイアスに基づき、シグモイド関数を使って確率的にニューロンの状態（0または1）を決定します。

---

### **状態の更新 (`sample_state` メソッド)**
```python
def sample_state(self, state, steps=1):
    """全ニューロンの状態をランダムに更新"""
    for _ in range(steps):
        for i in range(self.num_neurons):
            state[i] = self.sample_neuron(state, i)
    return state
```
- 複数回のステップにわたり、全てのニューロンをランダムに更新します。

---

### **学習 (`train` メソッド)**
```python
def train(self, data, epochs=1000, learning_rate=0.01):
    """ボルツマンマシンの学習"""
    num_samples = data.shape[0]
    for epoch in range(epochs):
        for sample in data:
            positive_state = sample.copy()
            negative_state = sample.copy()
            
            # 負方向のフェーズ（ギブスサンプリング）
            for _ in range(10):
                negative_state = self.sample_state(negative_state)
            
            # 重みとバイアスの更新
            for i in range(self.num_neurons):
                for j in range(self.num_neurons):
                    self.weights[i, j] += learning_rate * (
                        positive_state[i] * positive_state[j] - negative_state[i] * negative_state[j]
                    )
            self.biases += learning_rate * (positive_state - negative_state)
        
        # 温度を徐々に下げる（アニーリング）
        self.temperature *= 0.99
        
        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}/{epochs} completed, Temperature: {self.temperature:.4f}")
```
- **ポジティブフェーズ**: 入力データをそのまま使用。
- **ネガティブフェーズ**: ギブスサンプリングを使用して、ネットワークの現在の状態から新たなサンプルを生成。
- **重みとバイアスの更新**: ポジティブフェーズとネガティブフェーズの差異に基づいて学習。
- **アニーリング**: 温度を徐々に下げることで学習の安定性を向上。

---

### **データ生成 (`run` メソッド)**
```python
def run(self, state, steps=10):
    """学習済みボルツマンマシンでデータを生成"""
    return self.sample_state(state, steps)
```
- 学習したネットワークを使用して、与えられた初期状態から新しいパターンを生成します。

---

このクラスを使うことで、ボルツマンマシンを簡単に実装することができます。